In [1]:
import numpy as np
import pandas as pd

In [2]:
# crear DataFrame con 3 columnas y 500 filas
df = pd.DataFrame(np.random.randint(1, 101, size=(500, 3)), columns=['Parcial 1', 'Parcial 2', 'Parcial 3'])

# agregar cuarta columna con promedio de toda la fila
df['Promedio'] = df.mean(axis=1)

In [3]:
# mostrar primeras 3 filas del DataFrame
print(df.head(3))

   Parcial 1  Parcial 2  Parcial 3   Promedio
0         65         62         39  55.333333
1         60         32         84  58.666667
2         66         13         95  58.000000


In [4]:
cubierta = [(0, 40), (30, 70), (60, 100)]
cubierta

[(0, 40), (30, 70), (60, 100)]

In [5]:
def metrica(v, w):
    var = 0
    for i in range(0, 3):
        if var < abs(v[i] - w[i]):
            var = abs(v[i] - w[i])
    return var

In [6]:
df.iloc[1]

Parcial 1    60.000000
Parcial 2    32.000000
Parcial 3    84.000000
Promedio     58.666667
Name: 1, dtype: float64

In [7]:
df.iloc[0]

Parcial 1    65.000000
Parcial 2    62.000000
Parcial 3    39.000000
Promedio     55.333333
Name: 0, dtype: float64

In [8]:
metrica(df.iloc[1], df.iloc[0])

45.0